In [ ]:
import numpy as np
import cupy as cu
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib.animation import FuncAnimation
from montey import *

In [ ]:
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')
plt.rcParams["animation.html"] = "jshtml"

In [ ]:
def lognorm(arr):
    return colors.LogNorm(vmin=float(np.nanmin(np.where(arr > 0, arr, np.nan))), vmax=float(np.max(arr)))

In [ ]:
def time_animate(array, **plot_kwargs):
    fig, ax = plt.subplots(figsize=(12, 8), dpi=144)
    snorm = lognorm(array)
    ntof = len(array.coords["time"])

    mat = ax.pcolormesh(array.isel(time=0).values, norm=snorm, animated=True, **plot_kwargs)
    clb = fig.colorbar(mat)

    def animate(i):
        mat.set_array(array.isel(time=i).values)
        return mat,

    ani = FuncAnimation(fig, animate, frames=ntof, interval=100, blit=True)
    plt.close(fig)
    return ani

In [ ]:
# test = np.load("test.npz")
# ntof = test["photon_counter"].shape[-1]
# dt = 100
# test = xr.Dataset(
#         {
#             "Photons": (
#                 ["detector", "time"],
#                 test["photon_counter"],
#             ),
#             "Phi": (
#                 ["detector", "time"],
#                 test["phi_td"],
#                 {"long_name": "Φ"},
#             ),
#             "PhiPhase": (
#                 ["detector"],
#                 test["phi_phase"],
#                 {"units": "rad", "long_name": "Φ Phase"},
#             ),
#             "PhiDist": (
#                 ["detector", "time", "layer"],
#                 test["phi_dist"],
#                 {"long_name": "Φ Distribution"},
#             ),
#             "MomDist": (
#                 ["detector", "time", "layer"],
#                 test["mom_dist"],
#                 {"long_name": "Φ-Weighted Momentum Transfer Distribution"},
#             ),
#             "Fluence": (["x", "y", "z", "time"], test["fluence"]),
#         },
#         coords={
#             "time": (
#                 ["time"],
#                 (np.arange(ntof) + 0.5) * dt,
#                 {"units": "ps"}
#             ),
#         },
#     )
# test

In [ ]:
# tf = test["Fluence"].isel(z=100).sum(dim="time")
# tf.plot.pcolormesh(norm=lognorm(tf))

In [ ]:
# tf = test["Fluence"].isel(x=slice(0, 12), y=slice(89, 111), z=100).isel(time=2)
# tf.plot.pcolormesh(norm=lognorm(tf))

In [ ]:
# time_animate(test["Fluence"].isel(z=100))

In [ ]:
ndet = 80
det_area = 10**2 / 2
rhos = np.sqrt(det_area / np.pi * (1 + np.arange(ndet)))
rhos

In [ ]:
spec = Specification(
    nphoton=100,
    lifetime_max=5000,
    dt=100,
    lightspeed=0.2998,
    freq=110e-6,
)
source = Pencil(
    # position=Vector(100, 100, 0),
    position=Vector(0, 0, 0),
    direction=Vector(0, 0, 1)
)
n_media = 1.4
n_ext = n_media  # 1
states = [
    State(mua=0, mus=0, g=1, n=n_ext),
    State(mua=3e-2, mus=10, g=0.9, n=n_media),
    State(mua=2e-2, mus=12, g=0.9, n=n_media),
]
state = State(*zip(*(tuple(s.__dict__.values()) for s in states[1:])))

# media = np.ones((200, 200, 200), np.uint8)
# media[6:] = 2
# geom = VoxelGeometry(voxel_dim=(1.0, 1.0, 1.0), media_dim=media.shape)
media = np.array([0, 1, 2], dtype=np.uint8)
# geom = VoxelGeometry(voxel_dim=(1.0, 1.0, 1.0), media_dim=(200, 200, 200))
geom = AxialSymetricGeometry(voxel_dim=(0.1, 0.1), media_dim=(1000, 1000))
geom = LayeredGeometry(inner=geom, layers=(0, 6.5))

In [ ]:
res = monte_carlo(
    spec,
    source=source,
    states=states,
    detectors=[
        Detector(position=Vector(source.position.x, source.position.y, 0), radius=r) for r in rhos
    ],
    geom=geom,
    media=media,
)
for k, v in res.data_vars.items():
    v.data = cu.asnumpy(v.data)
res.coords["detector"] = rhos
res.coords["detector"].attrs["units"] = "mm"
for (dim, dv) in zip(res["Fluence"].dims[:-1], geom.inner.voxel_dim):
    res.coords[dim] = res.coords[dim] * dv
    res.coords[dim].attrs["units"] = "mm"
res.coords["time"].attrs["units"] = "ps"
res

In [ ]:
v = spec.lightspeed / n_media
omega = 2 * np.pi * spec.freq
pathLen = abs(res["PhiPhase"]) * v / omega
res["pathLen"] = pathLen * res["PhiDist"].sum(dim="time")
res["pathLen"].attrs["units"] = "mm"
res["pathLen"].attrs["long_name"] = "Φ-Weighted Path Length Distribution"

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(8, 8))
axs = axs.flat
res["Phi"].sum(dim="time").plot(ax=axs[0], yscale='log')
res["Photons"].sum(dim="time").plot(ax=axs[1], yscale='log', ylim=(1, None))
res["PhiPhase"].plot(ax=axs[2])
res["pathLen"].plot.line(ax=axs[3], x='detector')
fig.tight_layout();

In [ ]:
if res["Fluence"].ndim > 3:
    fluenceTimeSlice = res["Fluence"].isel(y=100)
else:
    fluenceTimeSlice = res["Fluence"]

In [ ]:
ssflu = fluenceTimeSlice.sum(dim="time")
ssflu.plot(norm=lognorm(ssflu))

In [ ]:
# ssflu2 = fluenceTimeSlice[90:110, 0:20].sum(dim="time")
ssflu2 = fluenceTimeSlice[:20, 0:20].sum(dim="time")
ssflu2.plot(norm=lognorm(ssflu2))

In [ ]:
# fluenceTimeSlice[90:110, 0:20].isel(time=1).plot()
fluenceTimeSlice[:20, 0:20].isel(time=1).plot()

In [ ]:
res["Phi"].plot(norm=lognorm(res["Phi"]))

In [ ]:
res["Photons"].plot(norm=lognorm(res["Photons"]))

In [ ]:
res["PhiDist"].plot.pcolormesh(norm=lognorm(res["PhiDist"]), col='layer')

In [ ]:
time_animate(fluenceTimeSlice)